In [8]:
import pandas as pd
import sqlite3 as sq

# Step 1: Load CSV Data

In [5]:
order = pd.read_csv("orders.csv")

In [7]:
print(order.head())

   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                  restaurant_name  
0               New Foods Chinese  
1  Ruchi Curry House Multicuisine  
2           Spice Kitchen Punjabi  
3          Darbar Kitchen Non-Veg  
4       Royal Eatery South Indian  


## Step 2: Load JSON Data

In [10]:
user = pd.read_json("users.json")
print(user.head())

   user_id    name       city membership
0        1  User_1    Chennai    Regular
1        2  User_2       Pune       Gold
2        3  User_3  Bangalore       Gold
3        4  User_4  Bangalore    Regular
4        5  User_5       Pune       Gold


# Step 3: Load SQL Data

In [11]:
connection = sq.connect("restaurants.db")
cursor = connection.cursor()

In [12]:
sql_file = open("restaurants.sql", "r")
sql = sql_file.read()
sql_file.close()

cursor.executescript(sql)
connection.commit()

In [14]:
restaurants = pd.read_sql("SELECT * FROM restaurants", connection)
print(restaurants.head())

   restaurant_id restaurant_name  cuisine  rating
0              1    Restaurant_1  Chinese     4.8
1              2    Restaurant_2   Indian     4.1
2              3    Restaurant_3  Mexican     4.3
3              4    Restaurant_4  Chinese     4.1
4              5    Restaurant_5  Chinese     4.8


# Step 4: Merge the Data

In [16]:
merge_df = pd.merge(
    order,
    user,
    on="user_id",
    how="left"
)

In [17]:
print(merge_df.head())

   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                  restaurant_name       name       city membership  
0               New Foods Chinese  User_2508  Hyderabad    Regular  
1  Ruchi Curry House Multicuisine  User_2693       Pune    Regular  
2           Spice Kitchen Punjabi  User_2084    Chennai       Gold  
3          Darbar Kitchen Non-Veg   User_319  Bangalore       Gold  
4       Royal Eatery South Indian  User_1064       Pune    Regular  


In [18]:
final = pd.merge(
    merge_df,
    restaurants,
    on="restaurant_id",
    how="left"
)

In [19]:
print(final.head())

   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                restaurant_name_x       name       city membership  \
0               New Foods Chinese  User_2508  Hyderabad    Regular   
1  Ruchi Curry House Multicuisine  User_2693       Pune    Regular   
2           Spice Kitchen Punjabi  User_2084    Chennai       Gold   
3          Darbar Kitchen Non-Veg   User_319  Bangalore       Gold   
4       Royal Eatery South Indian  User_1064       Pune    Regular   

  restaurant_name_y  cuisine  rating  
0    Restaurant_450  Mexican     3.2  
1    Restaurant_309   Indian     4.5  
2    Restaurant_107  Mexican     4.0  
3    Restaurant_224  Chinese    

# Step 5: Create Final Dataset

In [20]:
final.to_csv("final_food_delivery_dataset.csv", index=False)

## Which city has the highest total revenue (total_amount) from Gold members?  

In [27]:
df = pd.read_csv("final_food_delivery_dataset.csv")

gold = df[df["membership"] == "Gold"]

revenue = (
    gold
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

print(revenue.idxmax(), revenue.max())

Chennai 1080909.79


## Which cuisine has the highest average order value across all orders?

In [30]:
avg = (
    df
    .groupby("cuisine")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

print(avg.idxmax(), avg.max())

Mexican 808.0213444401395


## How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)?

In [31]:
total = (
    df
    .groupby("user_id")["total_amount"]
    .sum()
)

more_than = total[total > 1000]

user_count = more_than.count()

print(user_count)

2544


## Which restaurant rating range generated the highest total revenue?

In [70]:
# provide range/ rating range
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]
df["rating_range"] = pd.cut(
    df["rating"],
    bins=bins,
    labels=labels
)


In [33]:
rating_revenue = df.groupby("rating_range")["total_amount"].sum()
print(rating_revenue.idxmax())

4.6–5.0


C:\Users\tsart\AppData\Local\Temp\ipykernel_11952\1888971124.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  rating_revenue = df.groupby("rating_range")["total_amount"].sum()


## Among Gold members, which city has the highest average order value?

In [36]:
avg_city = gold.groupby("city")["total_amount"].mean()
print(avg_city.idxmax(), avg_city.max())

Chennai 808.4590800299178


## Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?

In [43]:
# count of unique cuisine
rest_count = (
    df.groupby("cuisine")["restaurant_id"]
      .nunique()
)

# total revenue by cuisine
cuisine_revenue = (
    df.groupby("cuisine")["total_amount"]
      .sum()
)

# final summary table of both 
cuisine_summary = pd.DataFrame({
    "distinct_restaurants": rest_count,
    "total_revenue": cuisine_revenue
})

# sort
cuisine_summary.sort_values("distinct_restaurants")


,distinct_restaurants,total_revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


## What percentage of total orders were placed by Gold members? (Rounded to nearest integer)

In [44]:
total_orders = len(df)
gold_orders = len(df[df["membership"] == "Gold"])

percentage = round((gold_orders / total_orders) * 100)

percentage

50

## Which restaurant has the highest average order value but less than 20 total orders?

In [49]:
restaurant_stats = (
    final.groupby("restaurant_name_x")
            .agg(
                avg_order_value=("total_amount", "mean"),
                total_orders=("order_id", "count")
            )
)

filtered = restaurant_stats[restaurant_stats["total_orders"] < 20]

filtered.sort_values("avg_order_value", ascending=False).head()


,avg_order_value,total_orders
restaurant_name_x,,
Hotel Dhaba Multicuisine,1040.222308,13
Sri Mess Punjabi,1029.180833,12
Ruchi Biryani Punjabi,1002.140625,16
Sri Delights Pure Veg,989.467222,18
Classic Kitchen Family Restaurant,973.167895,19


## Which combination contributes the highest revenue?

In [51]:
combo = (
    df.groupby(["membership", "cuisine"])["total_amount"].sum()
    
      .sort_values(ascending=False)
)

combo

membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

## During which quarter of the year is the total revenue highest?

In [54]:
final["order_date"] = pd.to_datetime(final["order_date"])

final["quarter"] = final["order_date"].dt.quarter

quarterly_revenue = (final.groupby("quarter")["total_amount"].sum().sort_values(ascending=False))

quarterly_revenue

quarter
3    2037385.10
4    2018263.66
1    2010626.64
2    1945348.72
Name: total_amount, dtype: float64

## How many total orders were placed by users with Gold membership?

In [57]:
gold = final[final["membership"] == "Gold"].shape[0]
gold

4987

## What is the total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city?

In [58]:
hyd = df[df["city"] == "Hyderabad"]["total_amount"].sum()
round(hyd)

1889367

## How many distinct users placed at least one order?

In [59]:
distinct_users = df["user_id"].nunique()
distinct_users

2883

## What is the average order value (rounded to 2 decimals) for Gold members?

In [64]:
gold = df[df["membership"] == "Gold"]["total_amount"].mean()
print(round(gold, 2))

797.15


## How many orders were placed for restaurants with rating ≥ 4.5?

In [65]:
orders_45 = df[df["rating"] >= 4.5].shape[0]
orders_45

3374

## How many orders were placed in the top revenue city among Gold members only?

In [68]:
gold_hyd_orders = final[(final["membership"] == "Gold") & (final["city"] == "Hyderabad")].shape[0]
gold_hyd_orders

1112

## The total number of rows in the final merged dataset is __________.

In [69]:
len(final)


10000